In [2]:
# IMPORTING LIBRARIES
import random
import numpy as np

# CONFIG SETUP
suits = ['C','D','H','S']
suits = ["♠","♣","♥","♦" ]
ranks  = {  "J":3,
            "9":2,
            "A":1,"10":1,
            "K":0,"Q":0
}
delim = "|"
num_players= 8
starting_player = 1

In [3]:
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.power = ranks[rank]

    def show(self):
        return self.rank + delim + self.suit

In [20]:
class Player:
    def __init__(self, play_n, init_hand):
        self.player_num = play_n
        self.hand = sorted(init_hand, key = lambda card: -1*card.power)

        self.team = play_n%2

    def strategy_1(self, round_cards, trump = ""):
        if len(round_cards)==0:
            ### OPENING HAND
            sorted_hand = sorted(self.hand, key = lambda card: -1*card.power)

            if sorted_hand[0].rank == 'J':
                # If player has a J then play it
                play = sorted_hand[0]
            else:
                # Else play the lowest hand
                play = sorted_hand[-1]
            

        else:
            ### ONGOING ROUND
            reqd_suit = round_cards[0][-1].suit
            sort_cards = sorted(round_cards, key = lambda card: -1*card[1].power)


            top_card = sort_cards[0]
            winning_team = (top_card[0]%2==0) 
            # and (top_card[-1].power >= 2)
            # print(f"Team {self.team} winning: {winning_team}")

            if winning_team==self.team: func = max
            else: func = min

            # print(sort_cards)
            valid_cards = [card for card in self.hand if card.suit==reqd_suit]

            if len(valid_cards)!=0:
                # You HAVE a valid Suit Card
                if winning_team:
                    play = valid_cards[0]
                else:
                    play = valid_cards[-1]
                    
            else:
                # You DO NOT HAVE a valid Suit Card
                j = [card for card in self.hand if card.power==3]
                nonj = [card for card in self.hand if card.power<3]
                if winning_team:
                    if len(nonj)>0:
                        play = nonj[0]
                    else:
                        play = j[0]


                else:
                    if len(nonj)==0:
                        
                        play = j[0]
                    else:
                        play = nonj[-1]
                    
        self.hand.remove(play)
        return play
    

In [21]:
class Game:
    
    def __init__(self, num_players, show = False):
        ### STARTING THE GAME BY DIRTRIBUTING CARDS

        # Getting all cards set up
        cards = [[Card(suit, rank) for suit in suits] for rank in ranks.keys()]
        # Converting List of Lists into a single List
        cards = [i for j in cards for i in j]

        if num_players >= 5: cards = cards*2

        
        # Shuffling the Cards
        random.shuffle(cards)
        
        # Splitting it to num_players in a dict
        self.players = {k+1: Player(k+1, list(hand)) for k, hand in enumerate(np.array_split(cards,num_players))}

        if show: self.show_hands()

        self.rounds = {}

    
    def show_hands(self):
        print("Current Hand of each Player")
        for player_num, player in self.players.items():
            print(str(player_num) + ") " + ", ".join([card.show() for card in player.hand]))
            

    def play_round(self, start_player=1):
        round_no = len(self.rounds.keys()) + 1
        print(f"\nRound {round_no}")

        round_stats = {}
        
        all_players = list(range(1,num_players+1))
        player_index = all_players.index(start_player)
        play_order = all_players[player_index:] + all_players[:player_index]

        self.round_cards = []
        for play in play_order:
            play_card = self.players[play].strategy_1(self.round_cards)
            print(str(play) + ") " + play_card.show())
            self.round_cards.append((play, play_card))

        self.rounds[round_no] = self.round_cards
            

In [23]:
game = Game(num_players, show = True)


for k in range(1,int(56/num_players)):
    game.play_round()
    game.show_hands()

Current Hand of each Player
1) J|♥, J|♦, 10|♥, A|♠, 10|♣, Q|♣
2) J|♠, 9|♠, A|♦, Q|♥, Q|♦, Q|♠
3) J|♣, J|♦, 10|♣, 10|♠, K|♠, K|♠
4) 9|♦, 9|♣, 9|♥, 10|♦, A|♥, A|♣
5) J|♥, 9|♠, 10|♥, K|♦, Q|♠, K|♣
6) J|♠, J|♣, 9|♦, A|♣, Q|♣, K|♣
7) 9|♥, A|♥, A|♦, 10|♠, K|♦, Q|♥
8) 9|♣, A|♠, 10|♦, K|♥, K|♥, Q|♦

Round 1
1) J|♥
2) Q|♥
3) K|♠
4) A|♥
5) 10|♥
6) K|♣
7) Q|♥
8) K|♥
Current Hand of each Player
1) J|♦, 10|♥, A|♠, 10|♣, Q|♣
2) J|♠, 9|♠, A|♦, Q|♦, Q|♠
3) J|♣, J|♦, 10|♣, 10|♠, K|♠
4) 9|♦, 9|♣, 9|♥, 10|♦, A|♣
5) J|♥, 9|♠, K|♦, Q|♠, K|♣
6) J|♠, J|♣, 9|♦, A|♣, Q|♣
7) 9|♥, A|♥, A|♦, 10|♠, K|♦
8) 9|♣, A|♠, 10|♦, K|♥, Q|♦

Round 2
1) J|♦
2) Q|♦
3) J|♦
4) 10|♦
5) K|♦
6) 9|♦
7) K|♦
8) Q|♦
Current Hand of each Player
1) 10|♥, A|♠, 10|♣, Q|♣
2) J|♠, 9|♠, A|♦, Q|♠
3) J|♣, 10|♣, 10|♠, K|♠
4) 9|♦, 9|♣, 9|♥, A|♣
5) J|♥, 9|♠, Q|♠, K|♣
6) J|♠, J|♣, A|♣, Q|♣
7) 9|♥, A|♥, A|♦, 10|♠
8) 9|♣, A|♠, 10|♦, K|♥

Round 3
1) Q|♣
2) Q|♠
3) 10|♣
4) A|♣
5) K|♣
6) Q|♣
7) 10|♠
8) 9|♣
Current Hand of each Player
1) 10|♥, A|♠, 10|♣
2